In [1]:
from transformers import pipeline
from datasets import load_dataset, Dataset
from huggingface_hub import login
import evaluate
import numpy as np

login()

checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use mps:0


In [2]:
drawings = load_dataset("imagefolder", data_dir="Drawings/")
ds = drawings['train'].to_pandas()

Resolving data files:   0%|          | 0/688 [00:00<?, ?it/s]

In [3]:
ds['image'] = ds['image'].apply(lambda x: x.get('path'))

In [ ]:
# Reference: https://huggingface.co/docs/transformers/en/tasks/zero_shot_image_classification
# Load evaluator
accuracy = evaluate.load("accuracy")

def compute_metrics(evals):
    predictions, labels = evals
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from PIL import Image

preds = []
labels = []

for index, row in ds.iterrows():
    image = Image.open(row['image'])
    pred = detector(image, candidate_labels=[0,1])
    prediction = pred[0]['label']
    preds.append(prediction)
    labels.append(row['label'])

In [6]:
accuracy.compute(predictions=preds, references=labels)

{'accuracy': 0.4738372093023256}